In [ ]:
from fastai.vision.all import*

In [ ]:
import torch
from torch import nn, optim
from torch.functional import F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import torchvision.models as models

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  
from sklearn.decomposition import PCA
import umap
import math
import io
import umap.plot
import plotly.graph_objs as go 
import plotly.io as pio 
pio.renderers.default ='iframe'


from collections import defaultdict
from tqdm import tqdm
import wandb
import random
from PIL import Image  # Add this import statement

%matplotlib inline
import math,os,sys
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
from Network import Network,Network_t ,ContrastiveLoss_with_margin, CustomVGG
from utils import init_weights, init_weights_for_gelu,initialize_weights_mod,plot_activation_stats

In [ ]:
import wandb
from fastai.callback.wandb import *
from fastai.data.core import DataLoaders

In [ ]:
device= "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device= "mps"

#device= "cpu" #overide device for overfitting a very small data batch
#net = net.to(device)

device

In [ ]:
from simple_cnn_dataset import TransformedDataset 
from mnist_dataset import MNISTDataset 
#load dataset 
data = pd.read_csv('../data/train.csv')
#data_shorted = data[:10000]
data_clp = data[:1000].reset_index(drop=True) # the index has to be readjusted otherwise it follows the whole data index... problematic
val_count = 200
#temporarly trying to overfit with less data


#common transformation
default_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5)
])

In [ ]:
train_dataset = MNISTDataset(data.iloc[:-val_count], default_transform)
val_dataset = MNISTDataset(data.iloc[-val_count:], default_transform)


# Define batch size and other DataLoader parameters
batch_size = 128 * 4
num_workers = 2

# Create DataLoaders
dls = DataLoaders.from_dsets(train_dataset, val_dataset, bs=batch_size, num_workers=num_workers)

In [ ]:
from fastai.learner import Learner
from fastai.callback.all import Callback
from fastai.learner import CancelStepException

class ContrastiveLossCallback(Callback):
    def __init__(self, loss_func):
        self.loss_func = loss_func

    def after_pred(self):
        embeddings1 = self.learn.pred[0]  # Assuming embeddings from the model
        embeddings2 = self.learn.pred[1]  # Assuming embeddings from the model
        distance = self.learn.y  # Assuming distance labels (0 or 1)

        loss = self.loss_func(embeddings1, embeddings2, distance)
        self.learn.loss = loss

In [ ]:
transformed_data = TransformedDataset(data_clp, default_transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(transformed_data))
val_size = len(transformed_data) - train_size
train_dataset, val_dataset = random_split(transformed_data, [train_size, val_size])

# Create DataLoaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [ ]:
#dls = DataLoaders(train_loader, val_loader)

In [ ]:
def get_x(row):
    # Assuming pixel values are in columns named 'pixel0', 'pixel1', etc.
    img = [row[f'pixel{i}'] for i in range(784)]  # Adjust 784 if your image size is different
    return torch.tensor(img).float().view(1,28, 28)  # Adjust 28, 28 to match your image dimensions

def get_y(row):
    return row['label']


dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=get_x,
    get_y=get_y,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    batch_tfms=default_transform
)

# Create the DataLoaders
#dls = dblock.dataloaders(data_clp, bs=64)

In [ ]:
#wandb.init(project="Contrastive_learning")


In [ ]:
model = CustomVGG()  # Initialize your custom model
loss_func = ContrastiveLoss_with_margin()  # Initialize your contrastive loss function

learner = Learner(dls, model, loss_func=loss_func, cbs=ContrastiveLossCallback(loss_func))
learner.model.to(device)

In [ ]:
learner.lr_find()

In [ ]:

def get_model(pretrained=False):
    return CustomVGG()
model = CustomVGG()
num_classes=10
dls.c = num_classes
#learner = vision_learner(dls=dls, arch= CustomVGG, metrics=error_rate)
# Create the learner directly
learner = Learner(dls, model, loss_func=nn.CrossEntropyLoss(), metrics=error_rate,cbs=WandbCallback())
learner.model.to(device)

In [ ]:
learner.fit(10, lr=0.00005)

In [ ]:
#learner.fine_tune(10)